In [1]:
import pandas as pd
import os
from PIL import Image
from matplotlib.pyplot import imshow
from shutil import copyfile

%matplotlib inline

# Join Datasets
Unions the aboutyou dataset and fashionID dataset together, based on the columns that they have in common.

In [2]:
data_path = '../../../data/'
df_about = pd.read_csv(os.path.join(data_path, 'aboutyou/img_attr.csv'), sep='\t', encoding='utf-8')
df_zalando = pd.read_csv(os.path.join(data_path, 'zalando/img_attr.csv'), sep='\t', encoding='utf-8')

In [3]:
common_cols = list(set(df_about.columns).intersection(set(df_zalando.columns)))
print(sorted(common_cols))

['ausschnitt_rückenausschnitt', 'ausschnitt_v-ausschnitt', 'category_hosen', 'category_jeans', 'category_kleider', 'category_roecke', 'category_shirts', 'color_beige', 'color_black', 'color_blue', 'color_gray', 'color_green', 'color_pink', 'color_red', 'color_white', 'color_yellow', 'img_path', 'länge_knielang', 'länge_normale länge', 'muster_gepunktet', 'muster_gestreift', 'passform_skinny', 'ärmellänge_dreiviertelarm', 'ärmellänge_halbarm', 'ärmellänge_langarm', 'ärmellänge_ärmellos']


In [4]:
col_names_mappings = {'ausschnitt': 'neckline',
                      'länge': 'length',
                      'ärmellänge': 'sleeves',
                      'passform': 'fit',
                      'muster': 'pattern',
                      'category': 'category',
                      'color': 'color'}

In [5]:
def rename_column_names(df_columns, mapping):
    renamed_cols = []
    
    for col in df_columns:
        col_name, col_value = col.split('_')
        col_name = mapping[col_name] if col_name in mapping else col_name
        renamed_cols.append(col_name + '_' + col_value)
        
    return renamed_cols

In [6]:
df_about.columns = rename_column_names(df_about.columns, col_names_mappings)
df_zalando.columns = rename_column_names(df_zalando.columns, col_names_mappings)

print(df_zalando.columns)
df_zalando.head()

Index(['img_path', 'category_blusen-tuniken', 'category_hosen',
       'category_jacken-maentel', 'category_jeans', 'category_kleider',
       'category_pullover-und-strickjacken', 'category_roecke',
       'category_shirts', 'color_beige', 'color_black', 'color_blue',
       'color_gray', 'color_green', 'color_pink', 'color_red', 'color_white',
       'color_yellow', 'sleeves_dreiviertelarm', 'sleeves_extrakurzer arm',
       'sleeves_extralanger arm', 'sleeves_halbarm', 'sleeves_kurzarm',
       'sleeves_langarm', 'sleeves_spaghettiträger', 'sleeves_ärmellos',
       'length_7/8 länge', 'length_extra kurz', 'length_extra lang',
       'length_knielang', 'length_knöchellang', 'length_kurz', 'length_lang',
       'length_normale länge', 'length_oberschenkellang', 'length_wadenlang',
       'pattern_geblümt', 'pattern_gepunktet', 'pattern_gestreift',
       'pattern_kariert', 'pattern_meliert', 'pattern_print',
       'pattern_unifarben', 'fit_flare', 'fit_körpernah', 'fit_loose fit',
 

,img_path,category_blusen-tuniken,category_hosen,category_jacken-maentel,category_jeans,category_kleider,category_pullover-und-strickjacken,category_roecke,category_shirts,color_beige,...,fit_tapered leg,fit_weit geschnitten,neckline_cache-coeur,neckline_carmen,neckline_rundhals,neckline_rückenausschnitt,neckline_tiefer rundhals,neckline_tiefer v-ausschnitt,neckline_u-boot,neckline_v-ausschnitt
0,pullover-und-strickjacken/0RA41G002-C11.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hosen/0VB21A009-B11.jpg,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,roecke/0VB21B007-K11.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,jacken-maentel/0VB21P000-G11.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,hosen/10K21A001-Q11.jpg,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
def unstack_columns(df, mappings):
    df_stacked = pd.DataFrame()
    
    for key, value in mappings.items():
        cols = [col for col in df.columns if value in col]
        df_col = pd.DataFrame()
        df_col[value] = df.set_index('img_path')[cols].idxmax(axis=1)
        df_col[value] = df_col[value].apply(lambda x: x.split('_')[-1])
        df_stacked = pd.concat([df_stacked, df_col], axis=1)

    return df_stacked

In [8]:
df_about = unstack_columns(df_about, col_names_mappings)
df_zalando = unstack_columns(df_zalando, col_names_mappings)

df_about.head()

,neckline,length,sleeves,fit,pattern,category,color
img_path,,,,,,,
hosen/00005b236646a41122e2af6608f9bc74.jpg,eingefasster ausschnitt,lang/maxi,dreiviertelarm,regular,all-over-muster,hosen,blue
kleider/000319a4556c44335bd4a97c8ebaa715.jpg,v-ausschnitt,kurz/mini,viertelarm,normale passform,all-over-muster,kleider,red
jeans/00036d763919b77400505f6534b7f353.jpg,eingefasster ausschnitt,lang/maxi,dreiviertelarm,skinny,all-over-muster,jeans,blue
hosen/00058a0b83bd94b4d4b5f12e12b239fe.jpg,eingefasster ausschnitt,lang/maxi,dreiviertelarm,slimfit,all-over-muster,hosen,green
strick/000610af30d05925ac9cfd0c28c763d6.jpg,rundhals-ausschnitt,normale länge,langarm,normale passform,all-over-muster,strick,gray


In [9]:
col_values_mappings = {
    'neckline': {
        'round': ['rundhals', 'tiefer rundhals', 'rundhals-ausschnitt'],
        'v': ['v-ausschnitt', 'tiefer v-ausschnitt', 'cache-coeur'],
        'back': ['rückenausschnitt'],
        'lined': ['eingefasster ausschnitt'],
        'wide': ['weiter ausschnitt', 'carmen', 'tiefer ausschnitt/dekolleté', 'u-boot', 'u-boot-ausschnitt']
    },
    'category': {
        'tops': ['shirts', 'tops'],
        'pants': ['hosen', 'jeans'],
        'jackets': ['jacken-maentel', 'jacken'],
        'dresses': ['kleider'],
        'skirts': ['roecke'],
        'knitwear': ['pullover-und-strickjacken', 'strick'],
        'blouses': ['blusen-tuniken', 'blusen-und-tuniken'],
        'jumpsuits': ['jumpsuits-und-overalls']
    },
    'length': {
        'short': ['extra kurz', 'kurz', 'kurz/mini', 'oberschenkellang', 'kurzer schnitt'],
        'normal': ['normale länge'],
        'knee': ['knielang'],
        '3-4': ['3/4-lang', '7/8 länge', '7/8-lang', 'wadenlang'],
        'long': ['knöchellang', 'extra lang', 'lang', 'lang/maxi', 'langer schnitt']
    },
    'pattern': {
        'floral': ['geblümt', 'geblümt/floral'],
        'polkadots': ['gepunktet'],
        'stripes': ['gestreift'],
        'print': ['kariert', 'meliert', 'print', 'all-over-muster'],
        'lace': ['spitze'],
        'unicolors': ['unifarben']
    },
    'fit': {
        'normal': ['flare', 'normal', 'straight leg', 'regular', 'normale passform'],
        'tight': ['körpernah', 'schmal', 'skinny', 'slim fit', 'tailiert', 
                  'figurbetonte passform', 'slimfit', 'schmale passform'],
        'loose': ['oversized', 'tapered leg', 'weit geschnitten', 'lockere passform']
    },
    'sleeves': {
        'short': ['extrakurzer arm', 'kurzarm', 'viertelarm'],
        'half': ['dreiviertelarm', 'halbarm'],
        'long': ['extralanger arm', 'langarm'],
        'sleeveless': ['spaghettiträger', 'ärmellos']
    }
}


In [10]:
def rename_column_values(df, col_values_mapping):
    for col_name, col_mappings in col_values_mappings.items():
        for col_key, col_values in col_mappings.items():
            df.loc[df[col_name].isin(col_values), col_name] = col_key
            
    return df

In [11]:
df_about = rename_column_values(df_about, col_values_mappings)
df_zalando = rename_column_values(df_zalando, col_values_mappings)

In [12]:
# check that the two dataframes have all columns in common
diff = list(set(df_about.columns).difference(set(df_zalando.columns)))
print(diff)

[]


## Create new folder with merged dataset

In [13]:
df_about['data_path'] = 'aboutyou'
df_zalando['data_path'] = 'zalando'
df = df_zalando.append(df_about)
print('Num data points', df.shape)
df.head()

Num data points (82783, 8)


,neckline,length,sleeves,fit,pattern,category,color,data_path
img_path,,,,,,,,
pullover-und-strickjacken/0RA41G002-C11.jpg,v,normal,long,normal,print,knitwear,gray,zalando
hosen/0VB21A009-B11.jpg,v,long,half,normal,unicolors,pants,beige,zalando
roecke/0VB21B007-K11.jpg,v,knee,half,normal,unicolors,skirts,blue,zalando
jacken-maentel/0VB21P000-G11.jpg,v,3-4,long,loose,unicolors,jackets,red,zalando
hosen/10K21A001-Q11.jpg,v,long,half,loose,floral,pants,black,zalando


In [14]:
df.loc['blusen-tuniken/2ET21E0L1-A11.jpg']

neckline              v
length           normal
sleeves      sleeveless
fit              normal
pattern       unicolors
category        blouses
color             white
data_path       zalando
Name: blusen-tuniken/2ET21E0L1-A11.jpg, dtype: object

In [15]:
merged_folder = os.path.join(data_path, 'fashion')
if not os.path.exists(merged_folder):
    os.makedirs(merged_folder)

In [ ]:
for idx, img_path in enumerate(df.index):
    if idx % 5000 == 0:
        print('Copied images: ', idx)
    
    src_path = os.path.join(data_path, df.loc[img_path, 'data_path'], img_path)
    dst_path = os.path.join(merged_folder, df.loc[img_path, 'category'], os.path.basename(img_path))
    
    dst_folder = os.path.dirname(dst_path)
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    copyfile(src_path, dst_path)

Change the img paths in the dataframe to match the new category folder structure.

In [19]:
df = df.drop(['data_path'], axis=1)
df = df.reset_index()
df['img_path'] = df['img_path'].apply(lambda x: os.path.basename(x))
df['img_path'] = df['category'] + '/' + df['img_path']
df = df.set_index('img_path')
df.head()

,neckline,length,sleeves,fit,pattern,category,color
img_path,,,,,,,
knitwear/0RA41G002-C11.jpg,v,normal,long,normal,print,knitwear,gray
pants/0VB21A009-B11.jpg,v,long,half,normal,unicolors,pants,beige
skirts/0VB21B007-K11.jpg,v,knee,half,normal,unicolors,skirts,blue
jackets/0VB21P000-G11.jpg,v,3-4,long,loose,unicolors,jackets,red
pants/10K21A001-Q11.jpg,v,long,half,loose,floral,pants,black


In [20]:
df_dum = pd.get_dummies(df)
df_dum.head()

,neckline_back,neckline_lined,neckline_round,neckline_v,neckline_wide,length_3-4,length_knee,length_long,length_normal,length_short,...,category_tops,color_beige,color_black,color_blue,color_gray,color_green,color_pink,color_red,color_white,color_yellow
img_path,,,,,,,,,,,,,,,,,,,,,
knitwear/0RA41G002-C11.jpg,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
pants/0VB21A009-B11.jpg,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
skirts/0VB21B007-K11.jpg,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
jackets/0VB21P000-G11.jpg,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
pants/10K21A001-Q11.jpg,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [24]:
df_stats = df_dum.copy()
new_cols = [tuple(col.split('_')) for col in df_dum.columns.values]
df_stats.columns = pd.MultiIndex.from_tuples(new_cols)
df_stats = df_stats.sum() / df_stats.shape[0] * 100
df_stats.to_csv(os.path.join(merged_folder, 'data_stats.csv'))
df_stats

neckline  back           1.641641
          lined         26.948770
          round         30.029112
          v             38.257855
          wide           3.122622
length    3-4           11.684766
          knee           8.830315
          long          20.176848
          normal        41.948226
          short         17.359844
sleeves   half          42.198277
          long          32.503050
          short         15.016368
          sleeveless    10.282304
fit       loose         10.479205
          loose fit      0.526678
          loosefit       0.684923
          normal        65.647536
          tailliert      4.579443
          tight         18.082215
pattern   floral        15.514055
          lace           2.675670
          polkadots      1.869949
          print         49.136900
          stripes        6.826281
          unicolors     23.977145
category  blouses       10.794487
          dresses       15.137166
          jackets        8.043922
          jump

In [27]:
df_dum.to_csv(os.path.join(merged_folder, 'img_attr.csv'), encoding='utf-8')